<a href="https://colab.research.google.com/github/fdsig/A-Lamp/blob/master/AVA_fb_vit_Image_Binary_Classification_aumentaions_subpatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import zipfile
import os
import random
from shutil import copyfile
import albumentations as A
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
!which python

/usr/local/bin/python


In [60]:
## to innitialize 
import io
import time
import subprocess
import sys
from tqdm import tqdm
def run_process (**kwrgs):
    logname = kwrgs['process']+'.log'
    with io.open(logname, 'wb') as writer, io.open(logname, 'rb', 1) as reader:
        process = subprocess.Popen(kwrgs['command'], stdout=writer, shell=True)
        while process.poll() is None:
            sys.stdout.write(reader.read())
            time.sleep(0.5)
        # Read the remaining
        sys.stdout.write(reader.read())

depend = [ 
                'torch==1.7.0', 'torchvision==0.8.1', 'timm==0.3.2', 
                'timm==0.3.2', 'albumentations==0.4.6'
                ]
for dep in depend:
    run_process(command='pip install '+dep, process = dep.split('==')[0])

cleanup = True
if cleanup:
    current = [fid for fid in os.scandir() if 'drive' not in fid.name]
    for i in current:
        os.system('rm -rf '+i.name)


#!git clone https://github.com/fdsig/convit.git

#!git clone https://github.com/facebookresearch/convit.git
repositories = ['https://github.com/mawady/convit.git',
                ' https://github.com/fdsig/image_utils',
                'https://github.com/fdsig/A-Lamp']
for repo in repositories:
    run_process(command='git clone '+repo, process=repo.split('/')[-1])

convit_fids = [fid for fid in os.scandir('convit/')]
other_fids = [fid for fid in os.scandir('image_utils/')]
for fid in convit_fids+other_fids:
    shutil.move(fid.path,fid.name)
os.system(
    'rm convit && rm image_utils && rm -rf convit &&\
     rm -rf MPADA && rm -rf image_utils')

import image_getter
pull = image_getter.Get_Ava()
pull.parse_urls()
pull.google_getter()
pull.ava_txt()
pull.download_ava_files(own_drive=True, 
                        download=True, 
                        full=True, 
                        clear_current=False)


https://drive.google.com/file/d/1YtU0m8cf2qgYcSpcPqlSz2GxO7wowu6W/view?usp=sharing


 The files are : ./batch_meta/ava_files_urls.txt
File IDs have sucessfully been obtaniend and now in content/batch_meta/
['1-9PbytN2awBviMP4Br8b4puhzFbUmOg2', '1-TcM8ZitdD-8yxnOlWDYcAIjszxDF5nH', '10hfdjtZgStNXMOYynLoIxlhiaItlAUQ4', '168mr0mkR6_5S5FoBkbzvUw8ulf-17D34', '16JJKoBivo0NUCcA_y2ueX2E0qFzWost7', '18ZMIqTlpfe7bm2cc6mgSkzfsxsmkeSTn', '1EimTfOAOnAsWLyzOESWVstKUcMJ6Ofqa', '1FRkogxsjjzaKumr-xYIp9rL_cLT01TR-', '1JkMJUIAyG3rn5eJ6VX8X5Z4Qq47UFqOi', '1KICR4a-a74aX6mSv9aAD3iXHV2n8vOtg', '1KQfOsOE1I5piQvEaVigYjw6do9Wn3Voz', '1M681Etlo_pot-p61f-wr8Uqf1J62XmuU', '1MifQhi1UJA255ulGYXz-_mWshDWdxrlv', '1NCJNl0gDHBC_cr-G026jIVrtrHgHcb62', '1NU2G7-qIJfg8OOB1KP0cZJ68arbrizJ-', '1NhkdKK5R1Cu-jg6VVMVCzKd9rFXXC9F_', '1OqB-CbF5iKqxgALFnqD6fOtILU0ow9vT', '1U1EvkK0NYlpWDZhRtMX5JzsKhSMO7iy2', '1UdGR02Be64mzuX276xlg--qvpiBa_jpL', '1W7cGe05y7FyB0yqvGiMnFAnGXpiSo1Cl', '1WXhlWVc_WX5GXLkyOhkHhS8kwxi9ko_7', '1_IO7KTXxV4xyOU

 20%|██        | 9/44 [01:06<04:55,  8.45s/it]Exception ignored in: <function ZipFile.__del__ at 0x7fc3e6128ef0>
Traceback (most recent call last):
  File "/usr/lib/python3.7/zipfile.py", line 1809, in __del__
    self.close()
  File "/usr/lib/python3.7/zipfile.py", line 1831, in close
    self._fpclose(fp)
  File "/usr/lib/python3.7/zipfile.py", line 1939, in _fpclose
    fp.close()
OSError: [Errno 107] Transport endpoint is not connected
100%|██████████| 255508/255508 [00:10<00:00, 24547.88it/s]


0 already exist 
 all files = 255508


45it [00:00, 4444.16it/s]


255508 [<DirEntry 'images'>]


In [61]:
#create desitination paths
root = 'data/'
test_train = train_dir,test_dir,val_dir = ['train/','test/', 'val/']
classes = class_1, class_0 = ['class_1/','class_0/']
for i in test_train:
    for j in classes:
        os.makedirs(root+i+j, exist_ok=True)

In [62]:
def meta_process():
    df = pd.read_csv('ava_meta_with_int_id_230721.csv')
    y_gt = df['mos_float'].values
    ids = df['ID'].values
    print(len(ids))
    y_gt_std, y_gt_mean = np.std(y_gt, axis = 0), np.mean(y_gt, axis = 0)
    exclude_below = y_gt_mean-y_gt_std*4
    exclude_above = y_gt_mean+y_gt_std*4
    ids = ids[np.argwhere(y_gt>=exclude_below)].ravel()
    y_gt = y_gt[np.argwhere(y_gt>=exclude_below)].ravel()
    print(len(y_gt))
    ids = ids[np.argwhere(y_gt<=exclude_above)].ravel()
    y_gt = y_gt[np.argwhere(y_gt<=exclude_above)].ravel()
    print(len(ids),len(y_gt))
    ids_low = ids[np.argwhere(y_gt<4.99)].ravel().astype(int)
    ids_high = ids[np.argwhere(y_gt>5.01)].ravel().astype(int)
    to_include = np.concatenate((ids_low,ids_high), axis=0)
    len(to_include)
    return df[df['ID'].isin(to_include)]

In [74]:
def create_dict():
    src_did = 'Images/images/'
    df = meta_process()
    y_g = df[['threshold','ID','set']].to_dict('index')
    y_g_dict = {str(y_g[pair_key]['ID']):y_g[pair_key]for pair_key in y_g}
    y_g_dict = {key:{**y_g_dict[key],**{'src':src_did+key+'.jpg'}} for key in y_g_dict} 
    y_g_dict = {key:{**y_g_dict[key], **{'dest':class_1}} if y_g_dict[key]['threshold']==1 
                else {**y_g_dict[key], **{'dest':class_0}} for key in y_g_dict }
    

def copy(subpatch=None, move=None):
    read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
    if move:
        for key in y_g_dict:
            if y_g_dict[key]['set'] == 'training':
                shutil.move(y_g_dict[key]['src'],root+train_dir+y_g_dict[key]['dest'])
                y_g_dict[key]['fid']=root+train_dir+y_g_dict[key]['dest']
            elif y_g_dict[key]['set'] == 'test':
                shutil.move(y_g_dict[key]['src'],root+test_dir+y_g_dict[key]['dest'])
                y_g_dict[key]['fid']=root+test_dir+y_g_dict[key]['dest']
            else:
                shutil.move(y_g_dict[key]['src'],root+val_dir+y_g_dict[key]['dest'])
                y_g_dict[key]['fid']=root+val_dir+y_g_dict[key]['dest']
    if copy:
        ##some process
         for key in y_g_dict:
            if y_g_dict[key]['set'] == 'training':
                y_g_dict[key]['fid']=root+test_dir+y_g_dict[key]['dest']
                img = read(y_g_dict[key]['src'])
                
            elif y_g_dict[key]['set'] == 'test':
                y_g_dict[key]['fid']=root+test_dir+y_g_dict[key]['dest']
                img = src = y_g_dict[key]['src']
                
            else:
                y_g_dict[key]['fid']=root+val_dir+y_g_dict[key]['dest']
                src = y_g_dict[key]['src']
                img = read(src)
                

    if subpatch:
        boxes_dict = get_patches()
        y_g_dict = for boxkey in y_g_dict
    
def get_patches():
    bbox_path = '/content/A-Lamp/multi_patch_subnet/adaptive_patch_selection/output-split'
    batches_dir = list(os.walk(bbox_path))[0][1]
    bbox_files = {i[-2:]:[j.path for j in os.scandir(bbox_path+'/'+i)] for i in batches_dir}
    keys = list(bbox_files.keys())
    for key in keys:
        for fl in bbox_files[key]:
            with open(fl, 'rb') as pk:
                if 'df' not in locals() or 'df' not in globals():
                    df = pd.read_pickle(pk)
                else:
                    df = df.append(pd.read_pickle(pk))
                
    
    return {k.split('/')[-1].split('.')[0]:{'patch':v} for k,v in zip(df['Filename'],df['BBoxes']) }



255502
255411
255403 255403


In [73]:
get_patches()

['chunkAT', 'chunkAW', 'chunkAI', 'chunkAZ', 'chunkAB', 'chunkAX', 'chunkAP', 'chunkAM', 'chunkAA', 'chunkBC', 'chunkAL', 'chunkAV', 'chunkAR', 'chunkAH', 'chunkBA', 'chunkAG', 'chunkAD', 'chunkAE', 'chunkBB', 'chunkAY', 'chunkAO', 'chunkAQ', 'chunkBD', 'chunkAS', 'chunkAK', 'chunkAC', 'chunkAU', 'chunkAJ', 'chunkAN', 'chunkAF']


{'721955': array([[385,   1, 609, 225],
        [217,  63, 441, 287],
        [269,  72, 493, 296],
        [  1, 156, 225, 380],
        [207, 161, 431, 385]]), '687431': array([[ 70,   9, 294, 233],
        [122,  92, 346, 316],
        [413,  75, 637, 299],
        [ 15, 348, 239, 572],
        [366, 335, 590, 559]]), '648887': array([[223,   4, 447, 228],
        [330,   1, 554, 225],
        [340, 254, 564, 478],
        [  5, 485, 229, 709],
        [260, 488, 484, 712]]), '702106': array([[ 11, 407, 235, 631],
        [ 37,   5, 261, 229],
        [ 63, 327, 287, 551],
        [204,  21, 428, 245],
        [227, 162, 451, 386]]), '696396': array([[  4,  10, 228, 234],
        [207,  62, 431, 286],
        [325,  77, 549, 301],
        [  5, 138, 229, 362],
        [ 14, 126, 238, 350]]), '871314': array([[212,  11, 436, 235],
        [479,  69, 703, 293],
        [  1, 135, 225, 359],
        [248, 307, 472, 531],
        [149, 301, 373, 525]]), '86868': array([[  0,   5, 224, 2

In [ ]:
from torchvision import datasets, transforms
import torchvision
from PIL import Image
from torchvision.utils import save_image
train_transforms = transforms.Compose(
    [
        Square(),
     transforms.Resize((224,224)),


        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(

    [
        Square(),
         transforms.Resize((224,224)),
        transforms.ToTensor(),
      torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
    ]

)


test_transforms = transforms.Compose(

    [   Square(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
      torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        )
     

    ]
)

In [ ]:
class Square:
    def __call__(self,img): 
            dims = np.array(img.size)
            m_x = dims[[dims.argmax()]][0]
            y_axis_pad = int(m_x-dims[1])//2 ;x_axis_pad = int(m_x-dims[0])//2
            y_axis_pad+=1;x_axis_pad+=1
            pad = (x_axis_pad, y_axis_pad, x_axis_pad, y_axis_pad)
            return torchvision.transforms.functional.pad(img,pad,0,'constant')
    
class Reflect:
    def __call__(self,img):
            a_transform = A.Compose([
            A.augmentations.transforms.LongestMaxSize(max_size=224),
            A.augmentations.transforms.PadIfNeeded(224,224,  always_apply=True),
            ])
            return a_transform(image=img)['image']
        
for key in y_g_dict:
    img_id = y_g_dict[key]['src'].split('/')[-1]
    if y_g_dict[key]['set'] == 'training':
        y_g_dict[key]['fid']=root+train_dir+y_g_dict[key]['dest']+img_id
    elif y_g_dict[key]['set'] == 'test':
        y_g_dict[key]['fid']=root+test_dir+y_g_dict[key]['dest']+img_id
    else:
        y_g_dict[key]['fid']=root+val_dir+y_g_dict[key]['dest']+img_id




In [ ]:
read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
for key in tqdm(y_g_dict):
    fid = y_g_dict[key]['fid']

    if 'image' in y_g_dict[key]:
        img = y_g_dict[key]['image']
        cv2.imwrite(fid,img)
        y_g_dict[key].pop('image')

    else:
        img = read(fid)
        zero_pad_array = train_transforms(Image.fromarray(img))
        save_image(zero_pad_array,fid)


100%|██████████| 253030/253030 [1:02:42<00:00, 67.25it/s]


In [ ]:
 fid_ = y_g_dict[list(y_g_dict.keys())[5]]['fid']
 read = lambda fid_: cv2.cvtColor(cv2.imread(fid_), cv2.COLOR_BGR2RGB)
 img = read(fid_)
 plt.imshow(img)

KeyError: ignored

In [ ]:
!python main.py  --pretrained --batch-size 200    --output_dir /content/drive/MyDrive/0.AVA/results/convit_augmenet_2_zero_pad/ --model convit_tiny --epochs 20 --data-path data/ --nb_classes 2 --nb_classes_pre 1000 


Not using distributed mode
Namespace(aa='rand-m9-mstd0.5-inc1', batch_size=200, clip_grad=None, color_jitter=0.4, cooldown_epochs=10, cutmix=1.0, cutmix_minmax=None, data_path='data/', data_set='IMNET', decay_epochs=30, decay_rate=0.1, device='cuda', dist_url='env://', distributed=False, drop=0.0, drop_block=None, drop_path=0.1, embed_dim=48, epochs=20, eval=False, inat_category='name', input_size=224, local_up_to_layer=10, locality_strength=1.0, lr=0.0005, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, min_lr=1e-05, mixup=0.8, mixup_mode='batch', mixup_prob=1.0, mixup_switch_prob=0.5, model='convit_tiny', model_ema=False, model_ema_decay=0.99996, model_ema_force_cpu=False, momentum=0.9, nb_classes=2, nb_classes_pre=1000, num_workers=10, opt='adamw', opt_betas=None, opt_eps=1e-08, output_dir='/content/drive/MyDrive/0.AVA/results/convit_augment_salient_crop/', patience_epochs=10, pin_mem=True, pretrained=True, recount=1, remode='pixel', repeated_aug=True, reprob=0.25, resplit=False

In [ ]:
# !python main.py --eval --model convit_tiny --resume /content/convit/models/checkpoint.pth --data-path /tmp/cats-v-dogs/ --nb_classes 2 --nb_classes_pre 1000

In [ ]:
#!python main.py  --output_dir /content/test --model convit_base --data-path data/ --nb_classes 2 --nb_classes_pre 1000 --